<a href="https://colab.research.google.com/github/ylmgrbzz/Future-Sales-Prediction-With-Machine-Learning-Random-Forest/blob/main/Future_Sales_Prediction_With_Machine_Learning_Random_Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
pip install catboost


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/base_command.py", line 180, in _main
    status = self.run(options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/req_command.py", line 199, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py", line 385, in run
    conflicts = self._determine_conflicts(to_install)
 

In [12]:
import numpy as np 
import pandas as pd
import os
from sklearn import *
import nltk,datetime
import datetime
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
import catboost
from catboost import Pool
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from xgboost import plot_importance
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [13]:
train=pd.read_csv("sales_train.csv")
test=pd.read_csv("test.csv")
shops=pd.read_csv("shops.csv")
submission=pd.read_csv("sample_submission.csv")
items=pd.read_csv("items.csv")
items_cats=pd.read_csv("item_categories.csv")



In [14]:
train.describe()

,date_block_num,shop_id,item_id,item_price,item_cnt_day
count,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06
mean,1.456991e+01,3.300173e+01,1.019723e+04,8.908532e+02,1.242641e+00
std,9.422988e+00,1.622697e+01,6.324297e+03,1.729800e+03,2.618834e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,-2.200000e+01
25%,7.000000e+00,2.200000e+01,4.476000e+03,2.490000e+02,1.000000e+00
50%,1.400000e+01,3.100000e+01,9.343000e+03,3.990000e+02,1.000000e+00
75%,2.300000e+01,4.700000e+01,1.568400e+04,9.990000e+02,1.000000e+00
max,3.300000e+01,5.900000e+01,2.216900e+04,3.079800e+05,2.169000e+03


In [15]:
test.shape

(214200, 3)

In [16]:
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [17]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 6 columns):
 #   Column          Dtype  
---  ------          -----  
 0   date            object 
 1   date_block_num  int64  
 2   shop_id         int64  
 3   item_id         int64  
 4   item_price      float64
 5   item_cnt_day    float64
dtypes: float64(2), int64(3), object(1)
memory usage: 134.4+ MB


In [18]:
train['date']=pd.to_datetime(train['date'],format='%d.%m.%Y')
train['month']=train['date'].dt.month
train['year']=train['date'].dt.year
train=train.drop(["date","item_price"],axis=1)
train=train.groupby([c for c in train.columns if c not in ['item_cnt_day']],as_index=False)[["item_cnt_day"]].sum()
train=train.rename(columns={"item_cnt_day":"item_cnt_month"})
train.head()

,date_block_num,shop_id,item_id,month,year,item_cnt_month
0,0,0,32,1,2013,6.0
1,0,0,33,1,2013,3.0
2,0,0,35,1,2013,1.0
3,0,0,43,1,2013,1.0
4,0,0,51,1,2013,2.0


In [19]:
shop_item_mean=train[["shop_id","item_id","item_cnt_month"]].groupby(["shop_id","item_id"],as_index=False)[["item_cnt_month"]].mean()


In [20]:
shop_item_mean=shop_item_mean.rename(columns={"item_cnt_month":"item_cnt_month_mean"})

In [23]:
shop_item_mean.head()

,shop_id,item_id,item_cnt_month_mean
0,0,30,31.0
1,0,31,11.0
2,0,32,8.0
3,0,33,3.0
4,0,35,7.5


In [24]:
train=pd.merge(train,shop_item_mean,how="left",on=["shop_id","item_id"])
train.head()

,date_block_num,shop_id,item_id,month,year,item_cnt_month,item_cnt_month_mean
0,0,0,32,1,2013,6.0,8.0
1,0,0,33,1,2013,3.0,3.0
2,0,0,35,1,2013,1.0,7.5
3,0,0,43,1,2013,1.0,1.0
4,0,0,51,1,2013,2.0,2.5


In [31]:
shop_prev_month=train[train['date_block_num']==33][["shop_id","item_id","item_cnt_month"]]

In [33]:
shop_prev_month=shop_prev_month.rename(columns={"item_cnt_month":"item_cnt_prev_month"})

In [34]:
shop_prev_month

,shop_id,item_id,item_cnt_prev_month
1577593,2,31,1.0
1577594,2,486,3.0
1577595,2,787,1.0
1577596,2,794,1.0
1577597,2,968,1.0
...,...,...,...
1609119,59,22087,6.0
1609120,59,22088,2.0
1609121,59,22091,1.0
1609122,59,22100,1.0


In [35]:
train=pd.merge(train,shop_prev_month,how="left",on=["shop_id","item_id"]).fillna(0.)

In [36]:
train=pd.merge(train,items,how="left",on="item_id")

In [37]:
train=pd.merge(train,items_cats,how="left",on="item_category_id")

In [39]:
train=pd.merge(train,shops,how="left",on="shop_id")

In [40]:
train.head()

,date_block_num,shop_id,item_id,month,year,item_cnt_month,item_cnt_month_mean,item_cnt_prev_month,item_name,item_category_id,item_category_name,shop_name
0,0,0,32,1,2013,6.0,8.0,0.0,1+1,40,Кино - DVD,"!Якутск Орджоникидзе, 56 фран"
1,0,0,33,1,2013,3.0,3.0,0.0,1+1 (BD),37,Кино - Blu-Ray,"!Якутск Орджоникидзе, 56 фран"
2,0,0,35,1,2013,1.0,7.5,0.0,10 ЛЕТ СПУСТЯ,40,Кино - DVD,"!Якутск Орджоникидзе, 56 фран"
3,0,0,43,1,2013,1.0,1.0,0.0,100 МИЛЛИОНОВ ЕВРО,40,Кино - DVD,"!Якутск Орджоникидзе, 56 фран"
4,0,0,51,1,2013,2.0,2.5,0.0,100 лучших произведений классики (mp3-CD) (Dig...,57,Музыка - MP3,"!Якутск Орджоникидзе, 56 фран"


In [41]:
test["month"]=11
test["year"]=2015
test["date_block_num"]=34

In [42]:
test=pd.merge(test,shop_item_mean,how="left",on=["shop_id","item_id"]).fillna(0.)

In [43]:
test=pd.merge(test,shop_prev_month,how="left",on=["shop_id","item_id"]).fillna(0.)

In [44]:
test=pd.merge(test,items,how="left",on="item_id")

In [45]:
test=pd.merge(test,items_cats,how="left",on="item_category_id")

In [46]:
test=pd.merge(test,shops,how="left",on="shop_id")

In [47]:
test['item_cnt_month']=0.
test.head()

,ID,shop_id,item_id,month,year,date_block_num,item_cnt_month_mean,item_cnt_prev_month,item_name,item_category_id,item_category_name,shop_name,item_cnt_month
0,0,5,5037,11,2015,34,1.444444,0.0,"NHL 15 [PS3, русские субтитры]",19,Игры - PS3,"Вологда ТРЦ ""Мармелад""",0.0
1,1,5,5320,11,2015,34,0.000000,0.0,ONE DIRECTION Made In The A.M.,55,Музыка - CD локального производства,"Вологда ТРЦ ""Мармелад""",0.0
2,2,5,5233,11,2015,34,2.000000,1.0,"Need for Speed Rivals (Essentials) [PS3, русск...",19,Игры - PS3,"Вологда ТРЦ ""Мармелад""",0.0
3,3,5,5232,11,2015,34,1.000000,0.0,"Need for Speed Rivals (Classics) [Xbox 360, ру...",23,Игры - XBOX 360,"Вологда ТРЦ ""Мармелад""",0.0
4,4,5,5268,11,2015,34,0.000000,0.0,"Need for Speed [PS4, русская версия]",20,Игры - PS4,"Вологда ТРЦ ""Мармелад""",0.0


In [48]:
for c in ['shop_name',"item_name","item_category_name"]:
  lbl=preprocessing.LabelEncoder()
  lbl.fit(list(train[c].unique())+list(test[c].unique()))
  train[c]=lbl.transform(train[c].astype(str))
  test[c]=lbl.transform(test[c].astype(str))
  print(c)

shop_name
item_name
item_category_name


In [49]:
col = [c for c in train.columns if c not in ["item_cnt_month"]]
x1=train[train["date_block_num"]<33]
y1=np.log1p(x1["item_cnt_month"].clip(0.,20.))
x1=x1[col]
x2=train[train['date_block_num']==33]
y2=np.log1p(x2["item_cnt_month"].clip(0.,20.))
x2=x2[col]

In [57]:
reg=ensemble.ExtraTreesRegressor(n_estimators=25,n_jobs=-1,max_depth=20,random_state=18)
reg.fit(x1,y1)


ExtraTreesRegressor(max_depth=20, n_estimators=25, n_jobs=-1, random_state=18)

In [58]:
print("RMSE VALUE İS:", np.sqrt(metrics.mean_squared_error(y2.clip(0.,20.),reg.predict(x2).clip(0.,20.))))

RMSE VALUE İS: 0.2884582171995078


In [54]:
reg.fit(train[col],train['item_cnt_month'].clip(0.,20.))
test["item_cnt_month"]=reg.predict(test[col]).clip(0.,20.)
test[["ID","item_cnt_month"]].to_csv("submission.csv",index=False)

test["item_cnt_month"]=np.expm1(test["item_cnt_month"])
test[["ID","item_cnt_month"]].to_csv("final_submission.csv",index=False)